In [ ]:
from pylab import *
import os
import sys
from mixture import GaussianMixture
import pickle
from PIL import Image

In [ ]:
dataset_dir = 'rgbd_dataset_freiburg3_long_office_household'
depth_dir = 'depth'
gmm_dir = 'gmm_qqvga_mine_quarterpixel_com2'

# og 
fx = 525.0  # focal length x
fy = 525.0  # focal length y
cx = 319.5  # optical center x
cy = 239.5  # optical center y

# fri3 
fx = 535.4  # focal length x
fy = 539.2  # focal length y
cx = 320.1  # optical center x
cy = 247.6  # optical center y
factor = 5000 # for the 16-bit PNG files
# OR: factor = 1 # for the 32-bit float images in the ROS bag files
baseline = 0.075 

full_depth_dir = os.path.join(dataset_dir,depth_dir)
full_gmm_dir = os.path.join(dataset_dir,gmm_dir)
if not os.path.exists(full_gmm_dir):
    os.mkdir(full_gmm_dir)

In [ ]:
max_disp = 325 #fx*baseline/0.125
min_disp = 2#f


In [ ]:
y_grid = np.repeat(np.arange(480)[:,None],640,1)
x_grid = np.repeat(np.arange(640)[None,:],480,0)

size_of_disp_error = 0.25 #0.125

def img_to_pts(img):
    Z = img/factor
    X = (x_grid-cx) * Z /fx
    Y = (y_grid-cy) * Z /fy
    
    d =  (fx*baseline)/Z
    zf = (fx*baseline)/np.clip(d+size_of_disp_error,min_disp,max_disp)
    zb = (fx*baseline)/np.clip(d-size_of_disp_error,min_disp,max_disp)
    
    subsample = 4
    step_frac = 0.5 * subsample
    Xp = x_grid-cx+step_frac
    Yp = y_grid-cy+step_frac
    Xm = x_grid-cx-step_frac
    Ym = y_grid-cy-step_frac
    frtptT = np.array(((Xp)/fx,(Yp)/fy,np.ones_like(d)))*zf
    lftptT = np.array(((Xm)/fx,(Yp)/fy,np.ones_like(d)))*Z
    rgtptT = np.array(((Xp)/fx,(Yp)/fy,np.ones_like(d)))*Z
    bckptT = np.array(((Xm)/fx,(Yp)/fy,np.ones_like(d)))*zb
    
    frtptB = np.array(((Xp)/fx,(Ym)/fy,np.ones_like(d)))*zf
    lftptB = np.array(((Xm)/fx,(Ym)/fy,np.ones_like(d)))*Z
    rgtptB = np.array(((Xp)/fx,(Ym)/fy,np.ones_like(d)))*Z
    bckptB = np.array(((Xm)/fx,(Ym)/fy,np.ones_like(d)))*zb
    
    box = np.stack([frtptT,lftptT,rgtptT,bckptT,frtptB,lftptB,rgtptB,bckptB])
    print(box.shape)
    
    m = box.mean(0)

    m1 = (m.reshape((3,-1)).T)[:,None]
    m2 = m1.reshape((-1,3,1))
    print(m1.shape)
    print(m2.shape)

    
    M = m2 @ m1
    print(M.shape)
    print(box.shape)
    
    covar = np.zeros( (640*480,3,3) )
    for pt in box:
        pt = pt.reshape((3,-1)).T
        pt1 = pt.reshape((-1,1,3))
        pt2 = pt.reshape((-1,3,1))

        covar += (pt2 @ pt1 - M)
    covar = covar/12
    covar = covar.reshape((480,640,3,3))[::subsample,::subsample,:,:]
    X = m[0][::subsample,::subsample]
    Y = m[1][::subsample,::subsample]
    Z = m[2][::subsample,::subsample]
    print(X.shape)

    xl = X[Z > 0]
    yl = Y[Z > 0]
    zl = Z[Z > 0]
    c1 = (covar.reshape((120,160,-1))[Z > 0]).reshape( (-1,3,3) )
    n = zl.shape[0]
    print(n,c1.shape)
    #print('MSHAPE\t',m.shape,Z.shape)
    idx = np.random.randint(n, size=n//16)

    return np.vstack([xl,yl,zl]).T, c1

def load_data(file):
    image = Image.open(file)   
    pixel = np.array(image)
    return img_to_pts(pixel)

for fl in sorted(os.listdir(full_depth_dir)):
    name,ext = os.path.splitext(fl)
    new_name = name + '.pkl'
    
    clf = GaussianMixture(100)
    data = load_data(os.path.join(full_depth_dir,fl))
    print([_.shape for _ in data])
    t1 = time.time()
    
    n = data[0].shape[0]
    clf.set_areas(np.ones(n))
    clf.set_covars(data[1])
    clf.fit(data[0])
    clf.set_areas(None)
    clf.set_covars(None)
    
    print(time.time()-t1)
    with open(os.path.join(full_gmm_dir,new_name),'wb') as fp:
        pickle.dump(clf,fp)
    

In [ ]:
frtptB.shape